In [1]:
import google.generativeai as genai
from db_utils import csv_read,query_database
import re

c:\mypython\project_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
genai.configure(api_key="xxxxx")

In [3]:
for model in genai.list_models():
   print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-exp-1206
models/gemini-exp-1121
models/gemini-exp-1114
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-experimental
models/embedding-001
models/text-embedding-004
models

In [4]:
def generate_sql_from_nl(user_input):
    csv_string = csv_read()
    system_prompt = f'''
        1. 你是一個財務資料庫專家，能夠生成相應的SQL指令。 
        2. SQL的編寫請用全英文。
        3. 非常重要!返回sql語法即可，不用其他說明與文字。
        4. MYSQL code中請不要出現資料庫中沒有的英文名稱。
        5. 請務必生成完整SQL語法，並一定要在首欄加入年或季度的欄位，例如seasonid。
       '''

    user_prompt = f'''
        1.資料庫結構、表名稱、欄位名稱請務必參考以下內容:{csv_string}
        2.一定不能生成沒有的sql表名與欄名，欄位名稱請盡量比對中文對照!
        3.請務必要判斷是否需要跨表查詢，如需要請使用join指令。
        4.另外時間只有年與季度seasonsID，例如2010年第一季格式為201001、2010年第二季格式為201002、
        2020年第三季格式為202003、2020年第四季格式為202004
                   '''
    model=genai.GenerativeModel(model_name="gemini-1.5-pro")

    prompt_content = [
        {"role": "user", "parts": [system_prompt]},
        {"role": "user", "parts": [user_input + user_prompt]}
    ]
    response = model.generate_content(
        contents = prompt_content,
        generation_config = genai.types.GenerationConfig(
            max_output_tokens=150,
            temperature=0.1
        )
    )
    sql_response = response.text
    print(f"回答:{sql_response}")
    return sql_response


模型可用系列:GPT、Gemini

測試生成sql，gemini pro生成結果比較精簡，速度快。gpt4omini比較能判斷複雜sql欄位，抓取正確欄位。

In [5]:
def extract_sql_queries(sql_response):
    matches = re.findall(r'SELECT.*?;', sql_response, re.IGNORECASE | re.DOTALL) 
    if matches:
        # 移除每個匹配項中的換行符號 
        matches = [match.replace('\n', ' ') for match in matches]
        return matches 
    else: 
        raise ValueError("SQL查詢指令未找到")

In [6]:
def main():
    user_input ="請給我2023年各季度之利息收入、法定盈餘公積、銷貨收入 "   #input("請輸入內容: ") 
    # 初次調用GPT判斷是否需要function calling

    model=genai.GenerativeModel(model_name="gemini-1.5-pro")
    prompt_content = [
        {"role": "user", 
         "parts":'''1.請判斷使用者的輸入是否需要連線到「財務報表」資料庫查詢，若需要請回答finvision即可!
                    2.若使用者的輸入是關於分析財務指標，請返回analysis即可!
                    3.若都不屬於查詢與分析範圍，只要回答NO即可!
                    4.財務報表資料庫內容包含資產負債表、損益表、現金流量表，其餘公司資料都沒有。'''},
        {"role": "user", "parts": user_input}
    ]
    
    
    response = model.generate_content(contents = prompt_content)
    
    
    print('===== 初次調用GPT判斷 =====')
    if 'finvision' in response.text:
        sql_response = generate_sql_from_nl(user_input) 
        sql_query = extract_sql_queries(sql_response)
        print(f'生成的SQL指令: {sql_query}') 
        # 執行GPT生成的SQL查詢 
        report = query_database(sql_query)
        print(f'查詢結果: {report}')
        return(f"finvision select :{report}")

    else: 
        # 一般對話處理
        return(response.text)
         


In [7]:
main()

===== 初次調用GPT判斷 =====
回答:```sql
SELECT
    s.seasonsID,
    SUM(pl.II) AS interest_income,
    SUM(bs.LR) AS legal_reserve,
    SUM(pl.NSR) AS sales_revenue
FROM
    pl
JOIN
    bs ON pl.seasonsID = bs.seasonsID
JOIN
    seasons s ON pl.seasonsID = s.seasonsID
WHERE
    s.seasonsID LIKE '2023%'
GROUP BY
    s.seasonsID;
```
生成的SQL指令: ["SELECT     s.seasonsID,     SUM(pl.II) AS interest_income,     SUM(bs.LR) AS legal_reserve,     SUM(pl.NSR) AS sales_revenue FROM     pl JOIN     bs ON pl.seasonsID = bs.seasonsID JOIN     seasons s ON pl.seasonsID = s.seasonsID WHERE     s.seasonsID LIKE '2023%' GROUP BY     s.seasonsID;"]
連線成功
查詢結果: [[{"seasonsID": "202301", "interest_income": 13617818.0, "legal_reserve": 311146899.0, "sales_revenue": 0.0}, {"seasonsID": "202302", "interest_income": 27289839.0, "legal_reserve": 311146899.0, "sales_revenue": 0.0}, {"seasonsID": "202303", "interest_income": 42885113.0, "legal_reserve": 311146899.0, "sales_revenue": 0.0}, {"seasonsID": "202304", "interest

'finvision select'